In [1]:
import convenience
df_train, sample_rates = convenience.load_train()
df_train.head()

,file_name,accent,gender,audio,length
0,1f_1018.wav,1,f,"[tensor(-0.0002), tensor(-6.1035e-05), tensor(...",3.413312
1,1f_1026.wav,1,f,"[tensor(-0.0008), tensor(-0.0008), tensor(-0.0...",5.632000
2,1f_1031.wav,1,f,"[tensor(-0.0002), tensor(-0.0002), tensor(-0.0...",4.693313
3,1f_1070.wav,1,f,"[tensor(0.0022), tensor(0.0019), tensor(0.0018...",3.925312
4,1f_1075.wav,1,f,"[tensor(0.), tensor(3.0518e-05), tensor(0.), t...",2.737500
